In [2]:
# Importe les modules
import numpy as np
import pandas as pd

# Projet 8 : Préparation des données

## Introduction

L'ensemble des données proviennent du Ministère de l'Education Nationale et ont été récupérées sur la plateforme [data.education.gouv.fr](https://data.education.gouv.fr/pages/accueil/). Les fichiers suivant ont été téléchargés au format CSV :
* `fr-en-ips-lycees-ap2022` ;
* `fr-en-ips_lycees` ;
* `fr-en-annuaire-education` ;
* `fr-en-indicateurs-de-resultat-des-lycees-denseignement-general-et-technologique` ;
* `fr-en-indicateurs-de-resultat-des-lycees-denseignement-professionnels`.

Tous ces fichiers ont été mis à jour en 2023 et ils contiennent des données jusqu'en 2022.

Un fichier supplémentaire, permettant à Power BI de créer des visuels de type « Carte de formes », a été généré à partir de [FranceGEOJSON](https://france-geojson.gregoiredavid.fr/).
> Le projet Github france-geojson propose au format GeoJSON les cartes des régions, départements, arrondissements, cantons et communes de France (métropole et départements d'outre-mer) à partir des données publiées par l'IGN et l'INSEE.

Le fichier `geojson` a ensuite été converti au format `json` grâce au site [Mapshaper](https://mapshaper.org/) pour être utilisable par Power BI. Il se nomme : `fr-departements-shape-map`.


## Objectif

Dans ce *notebook*, nous allons préparer les données qui vont être utilisées au sein de Power BI :
* les fichiers des IPS vont être concaténés ainsi que ceux des indicateurs de résulatats ;
* on va s'assurer que le périmètre géographique et temporel est cohérent entre fichiers ;
* on va procéder à différents nettoyage, renommage ou suppression des variables.

## Préparation des données

### Fichiers des IPS

In [3]:
df_ips_2022 = pd.read_csv('./datasets/raw/fr-en-ips-lycees-ap-2022.csv', sep=';')
df_ips_2016_2021 = pd.read_csv('./datasets/raw/fr-en-ips-lycees-2016-2021.csv', sep=';')

In [4]:
df_ips_2022.head(3)

,Rentrée scolaire,Académie,Code du département,Département,UAI,Nom de l'établissment,Code INSEE de la commune,Nom de la commune,Secteur,Type de lycée,Effectifs voie GT,Effectifs voie PRO,Effectifs Ensemble GT-PRO,IPS voie GT,IPS voie PRO,IPS Ensemble GT-PRO,Ecart-type de l'IPS voie GT,Ecart-type de l'IPS voie PRO
0,2022-2023,LYON,001,AIN,0010001W,LYCEE PROFESSIONNEL ALEXANDRE BERARD LYCEE DES...,01004,AMBERIEU EN BUGEY,public,LP,NaN,583.0,583.0,NaN,91.5,91.5,NaN,27.2
1,2022-2023,LYON,001,AIN,0010006B,LYCEE POLYVALENT SAINT EXUPERY LYCEE DES METIE...,01033,VALSERHONE,public,LPO,556.0,334.0,890.0,99.2,84.2,93.6,33.3,24.8
2,2022-2023,LYON,001,AIN,0010013J,LYCEE GENERAL LALANDE,01053,BOURG EN BRESSE,public,LEGT,1028.0,NaN,1028.0,123.0,NaN,123.0,34.7,NaN


In [5]:
df_ips_2016_2021.head(3)

,Rentrée scolaire,Académie,Code du département,Département,UAI,Nom de l'établissment,Code INSEE de la commune,Nom de la commune,Secteur,Type de lycée,IPS voie GT,IPS voie PRO,IPS Ensemble GT-PRO,Ecart-type de l'IPS voie GT,Ecart-type de l'IPS voie PRO
0,2018-2019,CRETEIL,77.0,SEINE-ET-MARNE,0771995A,LYCEE PROFESSIONNEL LE CHAMP DE CLAYE LYCEE DE...,77118.0,CLAYE SOUILLY,public,LP,NaN,87.4,87.4,NaN,NaN
1,2018-2019,CRETEIL,77.0,SEINE-ET-MARNE,0772127U,LYCEE GENERAL ET TECHNOLOGIQUE GALILEE,77122.0,COMBS LA VILLE,public,LEGT,115.6,NaN,115.6,NaN,NaN
2,2018-2019,CRETEIL,77.0,SEINE-ET-MARNE,0772151V,LYCEE PROFESSIONNEL PRIVE LES SINOPLIES,77390.0,ROISSY EN BRIE,privé sous contrat,LP,NaN,114.2,114.2,NaN,NaN


Les fichiers semblent correctement importés. On formate d'abord les noms des variables pour faciliter la suite des manipulations en s'aidant d'une fonction pour répéter facilement l'opération.

In [6]:
def snake_case (df):
    """
    Converti les noms de variables d'un DataFrame au format Snake Case :
    - mise en minuscule ;
    - remplacement de l'espaces, du tiret et de l'apostrophe par '_' ;
    - suppression des accents.
    """
    df.columns = (
        df.columns
        .str.lower()
        .str.replace('\'', '_')
        .str.replace(' - ', '_')
        .str.replace('-', '_')
        .str.replace(' ', '_')
        .str.normalize('NFKD')
        .str.encode('ascii', errors='ignore')
        .str.decode('utf-8')
    )

In [7]:
snake_case(df_ips_2016_2021)
snake_case(df_ips_2022)

In [8]:
print(
    f'Dimensions des DataFrames :\n'
    f'- df_ips_2022 : {df_ips_2022.shape}\n'
    f'- df_ips_2016_2021 : {df_ips_2016_2021.shape}'
    )

Dimensions des DataFrames :
- df_ips_2022 : (3598, 18)
- df_ips_2016_2021 : (21777, 15)


Le nombre de colonnes diffère, la dernière version contient plus de variables.

In [9]:
df_ips_2022.columns

Index(['rentree_scolaire', 'academie', 'code_du_departement', 'departement',
       'uai', 'nom_de_l_etablissment', 'code_insee_de_la_commune',
       'nom_de_la_commune', 'secteur', 'type_de_lycee', 'effectifs_voie_gt',
       'effectifs_voie_pro', 'effectifs_ensemble_gt_pro', 'ips_voie_gt',
       'ips_voie_pro', 'ips_ensemble_gt_pro', 'ecart_type_de_l_ips_voie_gt',
       'ecart_type_de_l_ips_voie_pro'],
      dtype='object')

In [10]:
df_ips_2016_2021.columns

Index(['rentree_scolaire', 'academie', 'code_du_departement', 'departement',
       'uai', 'nom_de_l_etablissment', 'code_insee_de_la_commune',
       'nom_de_la_commune', 'secteur', 'type_de_lycee', 'ips_voie_gt',
       'ips_voie_pro', 'ips_ensemble_gt_pro', 'ecart_type_de_l_ips_voie_gt',
       'ecart_type_de_l_ips_voie_pro'],
      dtype='object')

Les effectifs ont été ajoutés en 2022. On les supprime et on supprime également les colonnes des écarts-type.

In [11]:
df_ips_2022 = df_ips_2022.drop(
    columns=['effectifs_voie_gt', 'effectifs_voie_pro',
             'effectifs_ensemble_gt_pro', 'ecart_type_de_l_ips_voie_gt',
             'ecart_type_de_l_ips_voie_pro']
)

df_ips_2016_2021 = df_ips_2016_2021.drop(
    columns=['ecart_type_de_l_ips_voie_gt', 'ecart_type_de_l_ips_voie_pro']
)

In [12]:
df_ips_2022.dtypes

rentree_scolaire             object
academie                     object
code_du_departement          object
departement                  object
uai                          object
nom_de_l_etablissment        object
code_insee_de_la_commune     object
nom_de_la_commune            object
secteur                      object
type_de_lycee                object
ips_voie_gt                 float64
ips_voie_pro                float64
ips_ensemble_gt_pro         float64
dtype: object

In [13]:
df_ips_2016_2021.dtypes

rentree_scolaire             object
academie                     object
code_du_departement         float64
departement                  object
uai                          object
nom_de_l_etablissment        object
code_insee_de_la_commune    float64
nom_de_la_commune            object
secteur                      object
type_de_lycee                object
ips_voie_gt                 float64
ips_voie_pro                float64
ips_ensemble_gt_pro         float64
dtype: object

Les variables ci-dessous sont converties en catégorie :
* « code_du_departement » ;
* « code_insee_de_la_commune » ;
* « rentree_scolaire » ;
* « academie » ;
* « departement » ;
* « uai » ;
* « secteur » ;
* « type_de_lycee ».

In [14]:
df_ips_2022 = df_ips_2022.astype(
    {
        'code_du_departement': 'category',
        'code_insee_de_la_commune': 'category',
        'rentree_scolaire': 'category',
        'academie': 'category',
        'departement': 'category',
        'uai': 'category',
        'secteur': 'category',
        'type_de_lycee': 'category'
    }
)

df_ips_2016_2021 = df_ips_2016_2021.astype(
    {
        'code_du_departement': 'category',
        'code_insee_de_la_commune': 'category',
        'rentree_scolaire': 'category',
        'academie': 'category',
        'departement': 'category',
        'uai': 'category',
        'secteur': 'category',
        'type_de_lycee': 'category'
    }
)

In [15]:
df_ips_2022.isna().max()

rentree_scolaire            False
academie                    False
code_du_departement         False
departement                 False
uai                         False
nom_de_l_etablissment       False
code_insee_de_la_commune    False
nom_de_la_commune           False
secteur                     False
type_de_lycee               False
ips_voie_gt                  True
ips_voie_pro                 True
ips_ensemble_gt_pro         False
dtype: bool

Il y a des valeurs manquantes dans les IPS car, par exemple, un lycée professionnel ne peut pas avoir de valeurs d'IPS correspondant à la voie générale et inversement.

In [16]:
df_ips_2016_2021.isna().max()

rentree_scolaire            False
academie                    False
code_du_departement          True
departement                 False
uai                         False
nom_de_l_etablissment       False
code_insee_de_la_commune     True
nom_de_la_commune           False
secteur                     False
type_de_lycee               False
ips_voie_gt                  True
ips_voie_pro                 True
ips_ensemble_gt_pro         False
dtype: bool

On constate de nouveau les valeurs manquantes des IPS, par contre on a 90 valeurs manquantes dans « code_du_departement » et « code_insee_de_la_commune ».

In [17]:
df_ips_2016_2021.loc[df_ips_2016_2021['code_du_departement'].isna()].head(3)

,rentree_scolaire,academie,code_du_departement,departement,uai,nom_de_l_etablissment,code_insee_de_la_commune,nom_de_la_commune,secteur,type_de_lycee,ips_voie_gt,ips_voie_pro,ips_ensemble_gt_pro
143,2018-2019,CORSE,NaN,CORSE-DU-SUD,6200001G,LYCEE GENERAL ET TECHNOLOGIQUE FESCH AJACCIO,NaN,AJACCIO,public,LEGT,115.7,NaN,115.7
910,2019-2020,CORSE,NaN,CORSE-DU-SUD,6200650M,INSTITUTION SAINT PAUL (LYCEE GENERAL PRIVE) A...,NaN,AJACCIO,privé sous contrat,LEGT,113.6,NaN,113.6
911,2019-2020,CORSE,NaN,HAUTE-CORSE,7200093N,LYCEE PROFESSIONNEL JEAN NICOLI BASTIA,NaN,BASTIA,public,LP,NaN,85.3,85.3


In [18]:
df_ips_2016_2021.loc[
    df_ips_2016_2021['code_du_departement'].isna(), 'academie'].nunique()

1

Les valeurs sont manquantes uniquement pour l'académie de Corse. On crée un dictionnaire à partir de du fichier de 2022 et de la variables « uai », puis on utilise ce dernier pour compléter les valeurs dans le fichier de 2016 à 2022.

In [19]:
# Remplissage du code département
temp_dict = dict(zip(df_ips_2022['uai'], df_ips_2022['code_du_departement']))
df_ips_2016_2021['code_du_departement'] = (
    df_ips_2016_2021['code_du_departement']
    .astype('string')
    .fillna(df_ips_2016_2021['uai'].map(temp_dict))
)

# Remplissage du code commune
temp_dict = dict(zip(df_ips_2022['uai'], df_ips_2022['code_insee_de_la_commune']))
df_ips_2016_2021['code_insee_de_la_commune'] = (
    df_ips_2016_2021['code_insee_de_la_commune']
    .astype('string')
    .fillna(df_ips_2016_2021['uai'].map(temp_dict))
)

del temp_dict

La dernière étape avant de concaténer les deux tables est d'harmoniser le format des valeurs de « code_du_departement » et « code_insee_de_la_commune ». On souhaite obtenir :
* le code département avec deux caractères, à l'exception des DROM qui en compte trois ;
* le code de la commune avec cinq caractères.

En fonction de la source des données, le traitement sera différents.
|                          | `df_ips_2016_2021`                             | `df_ips_2022`          |
|--------------------------|:----------------------------------------------:|:----------------------:|
|`code_du_departement`     |Suppression décimale<br>Format deux caractères  |Format deux caractères  |
|`code_insee_de_la_commune`|Format deux caractères                          |Ok                      |

In [20]:
df_ips_2016_2021['code_du_departement'] = (
    df_ips_2016_2021['code_du_departement']
    .astype('string')
    .str[:-2] # Suppression décimale
    .apply(lambda x: '0' + x if len(x) == 1 else x) # Format 2 caractères
)

df_ips_2016_2021['code_insee_de_la_commune'] = (
    df_ips_2016_2021['code_insee_de_la_commune']
    .astype('string')
    .str[:-2] # Suppression décimale
)

df_ips_2022['code_du_departement'] = (
    df_ips_2022['code_du_departement']
    .astype('string')
    .apply(lambda x: x.strip()[1:] if x.startswith('0') else x) # Format 2 caractères
)

On peut concaténer les deux *DataFrames* et exporter le résultat en CSV. Comme l'étude porte sur la France métropolitaine on supprime également les enregistrements liés aux DROM-COM.

In [21]:
# Concaténe les données
df_temp = pd.concat([df_ips_2016_2021, df_ips_2022]).reset_index(drop=True)

# Met en majuscule le premier mot de "secteur"
df_temp['secteur'] = df_temp['secteur'].str.capitalize().astype('category')

# Crée la liste des départements à supprimer
liste_drom_com = list(
    df_temp.loc[
        (df_temp['code_du_departement'].str.startswith('9'))
        & (df_temp['code_du_departement'].str.len() == 3)
    ]['code_du_departement'].unique()
)

# Supprime les DROM-COM
df_temp = df_temp.loc[~df_temp['code_du_departement'].isin(liste_drom_com)]

# Exporte en CSV
df_temp.to_csv('./datasets/ips-lycees-2016-2022-hors-drom-com.csv', index=False
               , decimal=',' , quoting=1)

In [22]:
del(df_ips_2016_2021, df_ips_2022, df_temp, liste_drom_com)

## Fichier annuaire

In [23]:
df_annuaire = pd.read_csv('./datasets/raw/fr-en-annuaire-education.csv', sep=';', low_memory=False)

In [24]:
df_annuaire.head(3)

,Identifiant_de_l_etablissement,Nom_etablissement,Type_etablissement,Statut_public_prive,Adresse_1,Adresse_2,Adresse_3,Code postal,Code_commune,Nom_commune,...,rpi_concentre,rpi_disperse,code_nature,libelle_nature,Code_type_contrat_prive,PIAL,etablissement_mere,type_rattachement_etablissement_mere,code_bassin_formation,libelle_bassin_formation
0,0641141T,ECOLE PRIMAIRE PUBLIQUE HERAURITZ,Ecole,Public,Quartier Herauritz,NaN,64480 USTARITZ,64480,64547,Ustaritz,...,0.0,NaN,151,ECOLE DE NIVEAU ELEMENTAIRE,99,0641392R,NaN,NaN,04503,PAYS BASQUE
1,0641147Z,ECOLE PRIMAIRE DU BOURG,Ecole,Public,BOURG,NaN,64130 VIODOS ABENSE DE BAS,64130,64559,Viodos-Abense-de-Bas,...,0.0,NaN,151,ECOLE DE NIVEAU ELEMENTAIRE,99,0640039V,NaN,NaN,NaN,NaN
2,0641170Z,ECOLE ELEMENTAIRE PUBLIQUE,Ecole,Public,Rue de l'Ardoisière,NaN,64490 BEDOUS,64490,64104,Bedous,...,0.0,064411,151,ECOLE DE NIVEAU ELEMENTAIRE,99,0640016V,NaN,NaN,04502,CENTRE


In [25]:
df_annuaire.dtypes.to_frame().index

Index(['Identifiant_de_l_etablissement', 'Nom_etablissement',
       'Type_etablissement', 'Statut_public_prive', 'Adresse_1', 'Adresse_2',
       'Adresse_3', 'Code postal', 'Code_commune', 'Nom_commune',
       'Code_departement', 'Code_academie', 'Code_region', 'Ecole_maternelle',
       'Ecole_elementaire', 'Voie_generale', 'Voie_technologique',
       'Voie_professionnelle', 'Telephone', 'Fax', 'Web', 'Mail',
       'Restauration', 'Hebergement', 'ULIS', 'Apprentissage', 'Segpa',
       'Section_arts', 'Section_cinema', 'Section_theatre', 'Section_sport',
       'Section_internationale', 'Section_europeenne', 'Lycee_Agricole',
       'Lycee_militaire', 'Lycee_des_metiers', 'Post_BAC',
       'Appartenance_Education_Prioritaire', 'GRETA', 'SIREN_SIRET',
       'Nombre_d_eleves', 'Fiche_onisep', 'position', 'Type_contrat_prive',
       'Libelle_departement', 'Libelle_academie', 'Libelle_region',
       'coordonnee_X', 'coordonnee_Y', 'epsg', 'nom_circonscription',
       'latitude',

On commence par nettoyer les noms des variables.

In [26]:
snake_case(df_annuaire)

In [27]:
print(f'Dimensions du DataFrame : {df_annuaire.shape}')

Dimensions du DataFrame : (68962, 69)


Le fichier compte de nombreux enregistrements, hors, dans cet annuaire, plusieurs types d'établissements sont présents mais seuls les lycées nous intéressent.

In [28]:
df_annuaire['type_etablissement'].value_counts()

Ecole                         49973
Collège                        7649
Lycée                          5746
Médico-social                  2274
Service Administratif          2155
Information et orientation      426
Autre                           375
EREA                             79
Name: type_etablissement, dtype: int64

On ne garde que les lycées.

In [29]:
df_annuaire = df_annuaire.loc[df_annuaire['type_etablissement'] == 'Lycée'].reset_index()

De nombreuses variables sont également inutiles pour les analyses à venir. L'un des intérêts de cet annuaire est de savoir si un établissement appartient à une zone d'éducation prioritaire ou non. Les plupart des autres variables vont donc être inutiles.

In [30]:
df_annuaire.head(3)

,index,identifiant_de_l_etablissement,nom_etablissement,type_etablissement,statut_public_prive,adresse_1,adresse_2,adresse_3,code_postal,code_commune,...,rpi_concentre,rpi_disperse,code_nature,libelle_nature,code_type_contrat_prive,pial,etablissement_mere,type_rattachement_etablissement_mere,code_bassin_formation,libelle_bassin_formation
0,539,0741285V,Lycée professionnel privé ECA,Lycée,Privé,2 rue des Carillons,CS 70233,NaN,74942,74010,...,0.0,NaN,320,LYCEE PROFESSIONNEL,31,0740005D,NaN,NaN,08745,ALBANAIS ANNECIEN
1,540,0382893N,Lycée professionnel privé Jean-Marie Vianney,Lycée,Privé,22 avenue Hector Berlioz,BP 20,NaN,38260,38130,...,0.0,NaN,320,LYCEE PROFESSIONNEL,31,0382147C,NaN,NaN,08387,ISERE RHODANIENNE
2,541,0731040J,Lycée général privé Tomer Debora,Lycée,Privé,9 chemin de Saint-Pol,NaN,NaN,73100,73008,...,0.0,NaN,302,LYCEE D ENSEIGNEMENT GENERAL,30,NaN,NaN,NaN,08735,CHAMBERY


In [31]:
df_annuaire = df_annuaire[
    [
        'identifiant_de_l_etablissement',
        'nombre_d_eleves',
        'code_departement',
        'code_academie',
        'libelle_academie',
        'code_commune',
        'nom_commune',
        'restauration',
        'hebergement',
        'ulis',
        'apprentissage',
        'segpa',
        'section_arts',
        'section_cinema',
        'section_theatre',
        'section_sport',
        'section_internationale',
        'section_europeenne',
        'lycee_agricole',
        'lycee_militaire',
        'lycee_des_metiers',
        'post_bac',
        'etablissement_multi_lignes'
    ]
]

In [32]:
df_annuaire = df_annuaire.rename(
    columns={'identifiant_de_l_etablissement': 'uai'})

Certains établissements sont multi-lignes. Voyons à quoi cela correspond.

In [33]:
df_annuaire['etablissement_multi_lignes'].value_counts()

0    5610
1     136
Name: etablissement_multi_lignes, dtype: int64

In [34]:
df_annuaire.loc[df_annuaire['etablissement_multi_lignes'] == 1]['uai'].value_counts()

0580071M    3
0570279U    3
0690521S    3
0492015Z    2
0690574Z    2
           ..
9830418M    1
9870003P    1
9840265R    1
0342266D    1
9870025N    1
Name: uai, Length: 74, dtype: int64

In [35]:
df_annuaire.loc[df_annuaire['uai'] == '0580071M']

,uai,nombre_d_eleves,code_departement,code_academie,libelle_academie,code_commune,nom_commune,restauration,hebergement,ulis,...,section_cinema,section_theatre,section_sport,section_internationale,section_europeenne,lycee_agricole,lycee_militaire,lycee_des_metiers,post_bac,etablissement_multi_lignes
803,0580071M,660.0,058,7,Dijon,58194,Nevers,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1828,0580071M,660.0,058,7,Dijon,58194,Nevers,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
5239,0580071M,660.0,058,7,Dijon,58194,Nevers,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1


Les établissements sont dupliqués lorsqu'ils proposent des « services » supplémentaires. Pour supprimer les doublons, on groupe par UAI et, comme les services sont indiqués par « 0 » ou « 1 », on peut faire une somme pour garder l'information sur la ligne regroupée.

In [36]:
df_annuaire = df_annuaire.groupby('uai').agg(
    {
        'nombre_d_eleves': 'first',
        'code_departement': 'first',
        'code_academie': 'first',
        'libelle_academie': 'first',
        'code_commune': 'first',
        'nom_commune': 'first',
        'restauration': 'max',
        'hebergement': 'max',
        'ulis': 'max',
        'apprentissage': 'max',
        'segpa': 'max',
        'section_arts': 'max',
        'section_cinema': 'max',
        'section_theatre': 'max',
        'section_sport': 'max',
        'section_internationale': 'max',
        'section_europeenne': 'max',
        'lycee_agricole': 'max',
        'lycee_militaire': 'max',
        'lycee_des_metiers': 'max',
        'post_bac': 'max',
        'etablissement_multi_lignes': 'first'
    }
).reset_index()

On supprime la variable « etablissement_multi_lignes » devenue inutile.

In [37]:
df_annuaire = df_annuaire.drop(columns='etablissement_multi_lignes')

On enlève les DROM-COM.

In [38]:
# Crée la liste des départements à supprimer
liste_drom_com = list(
    df_annuaire.loc[
        (df_annuaire['code_departement'].str.startswith('9'))
        & (df_annuaire['code_departement'].str.len() == 3)
    ]['code_departement'].unique()
)

# Supprime les DROM-COM
df_annuaire = df_annuaire.loc[
    ~df_annuaire['code_departement'].isin(liste_drom_com)]

In [39]:
df_annuaire.isna().sum()

uai                          0
nombre_d_eleves           1915
code_departement             0
code_academie                0
libelle_academie             0
code_commune                 0
nom_commune                  0
restauration                20
hebergement                 20
ulis                        20
apprentissage              979
segpa                       20
section_arts               979
section_cinema             979
section_theatre            979
section_sport              979
section_internationale     979
section_europeenne         979
lycee_agricole             979
lycee_militaire            979
lycee_des_metiers           20
post_bac                   979
dtype: int64

In [40]:
taux_na = df_annuaire['nombre_d_eleves'].isna().mean()
print(f'{taux_na:.1%} des enregistrements n\'ont pas de valeur d\'effectifs.')
del taux_na

35.7% des enregistrements n'ont pas de valeur d'effectifs.


Le nombre d'établissements avec un effectif inconnu est trop important. On ne pourra pas mener d'analyses relatives à cette variables, on peut la supprimer.

In [41]:
df_annuaire = df_annuaire.drop(columns='nombre_d_eleves')

Pour les valeurs manquantes qui concernent les  informations d'hébergement, etc., on considère que si la valeur est manquante, on la met à 0.

In [42]:
df_annuaire = df_annuaire.fillna(0)

In [43]:
df_annuaire.dtypes

uai                        object
code_departement           object
code_academie               int64
libelle_academie           object
code_commune               object
nom_commune                object
restauration              float64
hebergement               float64
ulis                      float64
apprentissage             float64
segpa                     float64
section_arts              float64
section_cinema            float64
section_theatre           float64
section_sport             float64
section_internationale    float64
section_europeenne        float64
lycee_agricole            float64
lycee_militaire           float64
lycee_des_metiers         float64
post_bac                  float64
dtype: object

On change « uai », « code_departement », « code_academie », « libelle_academie » et « code_commune » en catégorie.

In [44]:
df_annuaire = df_annuaire.astype(
    {
        'uai': 'category',
        'code_departement': 'category',
        'code_academie': 'category',
        'libelle_academie': 'category',
        'code_commune': 'category'
    }
)

On formate le code département à 2 caractères.

In [45]:
df_annuaire['code_departement'] = (
    df_annuaire['code_departement']
    .astype('string')
    .apply(lambda x: x.strip()[1:] if x.startswith('0') else x) # Format 2 caractères
)

On exporte en CSV.

In [46]:
df_annuaire.to_csv('./datasets/annuaire-lycees-hors-drom-com.csv', index=False
               , decimal=',' , quoting=1)

In [47]:
del(liste_drom_com, df_annuaire)

### Fichiers des indicateurs de résultats

In [48]:
df_indicateurs_gt = pd.read_csv(
    './datasets/raw/fr-en-indicateurs-resultat-lycees-generaux-technologiques-2012-2022.csv'
    , sep=';', low_memory=False)
df_indicateurs_pro = pd.read_csv(
    './datasets/raw/fr-en-indicateurs-resultat-lycees-professionnels-2012-2022.csv'
    , sep=';', low_memory=False)

In [49]:
df_indicateurs_gt.head(3)

,Etablissement,Annee,Ville,UAI,Code commune,Academie,Departement,Secteur,Presents - L,Presents - ES,...,Taux de mentions - Gnle,Valeur ajoutée du taux de mentions - Gnle,Nombre de mentions TB avec félicitations - G,Nombre de mentions TB sans félicitations - G,Nombre de mentions B - G,Nombre de mentions AB - G,Nombre de mentions TB avec félicitations - T,Nombre de mentions TB sans félicitations - T,Nombre de mentions B - T,Nombre de mentions AB - T
0,LYCEE CAMILLE CLAUDEL,2017,BLOIS,0410959V,41018,ORLEANS-TOURS,LOIR ET CHER,public,35.0,60.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LYCEE ALBERT CAMUS,2017,FIRMINY,0420013L,42095,LYON,LOIRE,public,35.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LYCEE GEORGES BRASSENS (GENERAL ET TECHNO.),2017,RIVE DE GIER,0420027B,42186,LYON,LOIRE,public,27.0,64.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
df_indicateurs_pro.head(3)

,Etablissement,UAI,Annee,Ville,Code commune,Academie,Departement,Secteur,Presents - Specialites pluri-technologiques de la production,Presents - Transformations,...,Valeur ajoutee du taux de mentions - Specialites plurivalentes des services,Valeur ajoutee du taux de mentions - Echanges et gestion,Valeur ajoutee du taux de mentions - Communication et information,Valeur ajoutee du taux de mentions - Services aux personnes,Valeur ajoutee du taux de mentions - Services a la collectivite,Valeur ajoutee du taux de mentions - Services,Valeur ajoutee du taux de mentions - Toutes series,Nombre de mentions TB sans félicitations - P,Nombre de mentions B - P,Nombre de mentions AB - P
0,LYCEE CHRISTOPHE COLOMB (PROFESSIONNEL),0941918Z,2015,SUCY EN BRIE,94071,CRETEIL,VAL-DE-MARNE,public,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LYCEE FERNAND LEGER (PROFESSIONNEL),0941972H,2015,IVRY SUR SEINE,94041,CRETEIL,VAL-DE-MARNE,public,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LYCEE PROFESSIONNEL VIRGINIA HENDERSON,0950709E,2015,ARNOUVILLE,95019,VERSAILLES,VAL D'OISE,public,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
snake_case(df_indicateurs_gt)
snake_case(df_indicateurs_pro)

In [ ]:
df_indicateurs_gt = df_indicateurs_gt[
    [
        'etablissement',
        'annee',
        'ville',
        'uai',
        'code_commune',
        'academie',
        'departement',
        'secteur',
        'presents_l',
        'presents_es',
        'presents_s',
        'presents_stg',
        'presents_sti2d',
        'presents_std2a',
        'presents_stmg',
        'presents_sti',
        'presents_stl',
        'presents_st2s',
        'presents_tmd',
        'presents_sthr',
        'presents_toutes_series',
        'taux_de_reussite_l',
        'taux_de_reussite_es',
        'taux_de_reussite_s',
        'taux_de_reussite_stg',
        'taux_de_reussite_sti2d',
        'taux_de_reussite_std2a',
        'taux_de_reussite_stmg',
        'taux_de_reussite_sti',
        'taux_de_reussite_stl',
        'taux_de_reussite_st2s',
        'taux_de_reussite_tmd',
        'taux_de_reussite_sthr',
        'taux_de_reussite_toutes_series',
        'taux_de_reussite_attendu_acad_l',
        'taux_de_reussite_attendu_acad_es',
        'taux_de_reussite_attendu_acad_s',
        'taux_de_reussite_attendu_acad_stg',
        'taux_de_reussite_attendu_acad_sti2d',
        'taux_de_reussite_attendu_acad_std2a',
        'taux_de_reussite_attendu_acad_stmg',
        'taux_de_reussite_attendu_acad_sti',
        'taux_de_reussite_attendu_acad_stl',
        'taux_de_reussite_attendu_acad_st2s',
        'taux_de_reussite_attendu_acad_tmd',
        'taux_de_reussite_attendu_acad_sthr',
        'taux_de_reussite_attendu_acad_toutes_series',
        'taux_de_reussite_attendu_france_l',
        'taux_de_reussite_attendu_france_es',
        'taux_de_reussite_attendu_france_s',
        'taux_de_reussite_attendu_france_stg',
        'taux_de_reussite_attendu_france_sti2d',
        'taux_de_reussite_attendu_france_std2a',
        'taux_de_reussite_attendu_france_stmg',
        'taux_de_reussite_attendu_france_sti',
        'taux_de_reussite_attendu_france_stl',
        'taux_de_reussite_attendu_france_st2s',
        'taux_de_reussite_attendu_france_tmd',
        'taux_de_reussite_attendu_france_sthr',
        'taux_de_reussite_attendu_france_toutes_series',
        'taux_de_mentions_l',
        'taux_de_mentions_es',
        'taux_de_mentions_s',
        'taux_de_mentions_sti2d',
        'taux_de_mentions_std2a',
        'taux_de_mentions_stmg',
        'taux_de_mentions_stl',
        'taux_de_mentions_st2s',
        'taux_de_mentions_tmd',
        'taux_de_mentions_sthr',
        'taux_de_mentions_toutes_series',
        'taux_de_mentions_attendu_l',
        'taux_de_mentions_attendu_es',
        'taux_de_mentions_attendu_s',
        'taux_de_mentions_attendu_sti2d',
        'taux_de_mentions_attendu_std2a',
        'taux_de_mentions_attendu_stmg',
        'taux_de_mentions_attendu_stl',
        'taux_de_mentions_attendu_st2s',
        'taux_de_mentions_attendu_tmd',
        'taux_de_mentions_attendu_sthr',
        'taux_de_mentions_attendu_toutes_series',
        'structure_pedagogique_5_groupes',
        'structure_pedagogique_7_groupes',
        'pourcentage_bacheliers_sortants_2de_1re_terminale_etablissement',
        'pourcentage_bacheliers_sortants_terminales_etablissement',
        'pourcentage_bacheliers_sortants_2de_1re_terminale_acad',
        'pourcentage_bacheliers_sortants_terminale_acad',
        'pourcentage_bacheliers_sortants_2de_1re_terminale_france',
        'pourcentage_bacheliers_sortants_terminale_france',
        'effectif_de_seconde',
        'effectif_de_premiere',
        'effectif_de_terminale',
        'taux_d_acces_2nde_bac',
        'taux_d_acces_attendu_acad_2nde_bac',
        'taux_d_acces_attendu_france_2nde_bac',
        'taux_d_acces_1ere_bac',
        'taux_d_acces_attendu_acad_1ere_bac',
        'taux_d_acces_attendu_france_1ere_bac',
        'taux_d_acces_terminale_bac',
        'taux_d_acces_attendu_france_terminale_bac',
        'region',
        'code_region',
        'code_departement',
        'libelle_departement',
        'valeur_ajoutee_du_taux_de_reussite_toutes_series',
        'valeur_ajoutee_du_taux_d_acces_2nde_bac',
        'valeur_ajoutee_du_taux_de_mentions_toutes_series',
        'valeur_ajoutee_du_taux_de_reussite_l',
        'valeur_ajoutee_du_taux_de_reussite_es',
        'valeur_ajoutee_du_taux_de_reussite_s',
        'valeur_ajoutee_du_taux_de_reussite_stg',
        'valeur_ajoutee_du_taux_de_reussite_sti2d',
        'valeur_ajoutee_du_taux_de_reussite_std2a',
        'valeur_ajoutee_du_taux_de_reussite_stmg',
        'valeur_ajoutee_du_taux_de_reussite_sti',
        'valeur_ajoutee_du_taux_de_reussite_stl',
        'valeur_ajoutee_du_taux_de_reussite_st2s',
        'valeur_ajoutee_du_taux_de_reussite_tmd',
        'valeur_ajoutee_du_taux_de_reussite_sthr',
        'valeur_ajoutee_du_taux_d_acces_1ere_bac',
        'valeur_ajoutee_du_taux_d_acces_terminale_bac',
        'valeur_ajoutee_du_taux_de_mentions_l',
        'valeur_ajoutee_du_taux_de_mentions_es',
        'valeur_ajoutee_du_taux_de_mentions_s',
        'valeur_ajoutee_du_taux_de_mentions_sti2d',
        'valeur_ajoutee_du_taux_de_mentions_std2a',
        'valeur_ajoutee_du_taux_de_mentions_stmg',
        'valeur_ajoutee_du_taux_de_mentions_stl',
        'valeur_ajoutee_du_taux_de_mentions_st2s',
        'valeur_ajoutee_du_taux_de_mentions_tmd',
        'valeur_ajoutee_du_taux_de_mentions_sthr',
        'presents_gnle',
        'taux_de_reussite_gnle',
        'valeur_ajoutee_du_taux_de_reussite_gnle',
        'taux_de_mentions_gnle',
        'valeur_ajoutee_du_taux_de_mentions_gnle',
        'nombre_de_mentions_tb_avec_felicitations_g',
        'nombre_de_mentions_tb_sans_felicitations_g',
        'nombre_de_mentions_b_g',
        'nombre_de_mentions_ab_g',
        'nombre_de_mentions_tb_avec_felicitations_t',
        'nombre_de_mentions_tb_sans_felicitations_t',
        'nombre_de_mentions_b_t',
        'nombre_de_mentions_ab_t'
    ]
]